In [1]:
import os
import pandas as pd
import re
import numpy as np
import tensorflow as tf

2024-02-09 22:24:46.184376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [182]:
from OneHotEncoding import get_one_hot_sequence
from BuildMatrixFromDotBracket import get_couples, build_matrix

In [194]:
example_2_structure = "...(((((((..((((((.........))))))......).((((((.......))))))..))))))..."
example_2_sequence = "CGCUUCAUAUAAUCCUAAUGAUAUGGUUUGGGAGUUUCUACCAAGAGCCUUAAACUCUUGAUUAUGAAGUG"
example_2_length = len(example_2_sequence)

# Get one-hot encoded sequence (X) and adjacency matrix (y) of example data
example_2_one_hot = get_one_hot_sequence(example_2_sequence)
example_2_matrix = build_matrix(get_couples(example_2_structure), example_2_length)

# Creating the model

In [188]:
# Define the input layer to accept one-hot encoded sequences
input_layer = tf.keras.layers.Input(shape=(example_2_length, 4))

In [189]:
# Process sequences using an LSTM layer, returning all outputs
lstm_layer = tf.keras.layers.LSTM(units=64, return_sequences=True)(input_layer)

In [190]:
# Flatten the LSTM output into a 1D vector
flattened = tf.keras.layers.Flatten()(lstm_layer)

In [191]:
# Create a dense output layer with sigmoid activation for probabilities
output_layer = tf.keras.layers.Dense(example_2_length * example_2_length, activation='sigmoid')(flattened)

In [192]:
# Reshape the output into a 2D adjacency matrix
adjacency_matrix = tf.reshape(output_layer, (-1, example_2_length, example_2_length))

In [193]:
# Construct the model, specifying input and output layers
model = tf.keras.Model(inputs=input_layer, outputs=adjacency_matrix)

In [177]:
# Configure the model for training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [178]:
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 71, 4)]           0         
                                                                 
 lstm_17 (LSTM)              (None, 71, 64)            17664     
                                                                 
 flatten_13 (Flatten)        (None, 4544)              0         
                                                                 
 dense_14 (Dense)            (None, 5041)              22911345  
                                                                 
 tf.reshape_13 (TFOpLambda)  (None, 71, 71)            0         
                                                                 
Total params: 22929009 (87.47 MB)
Trainable params: 22929009 (87.47 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [180]:
# Reshape input data to match model's expected shape
example_2_one_hot = example_2_one_hot.reshape(1, 71, 4)
print(example_2_one_hot.shape, example_2_matrix.shape)

(1, 71, 4) (1, 71, 71)


In [181]:
# Train the model with input and target data for 10 epochs
model.fit(example_2_one_hot, example_2_matrix, epochs=10)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 208ms/step - loss: 0.6281 - accuracy: 0.4366
Epoch 3/10
1/1 [==============================] - 0s 318ms/step - loss: 0.5551 - accuracy: 0.4366
Epoch 4/10
1/1 [==============================] - 0s 194ms/step - loss: 0.4765 - accuracy: 0.4225
Epoch 5/10
1/1 [==============================] - 0s 241ms/step - loss: 0.3943 - accuracy: 0.4225
Epoch 6/10
1/1 [==============================] - 0s 194ms/step - loss: 0.3108 - accuracy: 0.4366
Epoch 7/10
1/1 [==============================] - 0s 234ms/step - loss: 0.2295 - accuracy: 0.4366
Epoch 8/10
1/1 [==============================] - 0s 200ms/step - loss: 0.1547 - accuracy: 0.4225
Epoch 9/10
1/1 [==============================] - 0s 243ms/step - loss: 0.0916 - accuracy: 0.4085
Epoch 10/10
1/1 [==============================] - 0s 186ms/step - loss: 0.0451 - accuracy: 0.3944
